In [19]:
import matplotlib.pyplot as plt
import networkx as nx

from collections import Counter
from gensim.models.phrases import Phrases, Phraser
from hvectorspaces.io import PostgresClient

In [86]:
list(range(1930,2030,10))

[1930, 1940, 1950, 1960, 1970, 1980, 1990, 2000, 2010, 2020]

In [87]:
fields = [
    "title", 
    "publication_year", 
    "domain", 
    "field", 
    "topic",
    "abstract"
]

data_per_decade = list()
decades = list(range(1920,2030,10))

with PostgresClient() as client:
    for d in decades:
        citation_map = client.fetch_per_decade_data(d, additional_fields=fields)
        id_to_cited_ids = {}
        id_to_attrs = {} 
        for oa_id, refs, title, publication_year, domain, field, topic, abstract in citation_map:
            id_to_cited_ids[oa_id] = refs
            id_to_attrs[oa_id] = {
                "title": title,
                "publication_year": publication_year,
                "domain": domain,
                "field": field,
                "topic": topic,
                "abstract" : abstract
            }
        data_per_decade.append(id_to_attrs)

In [88]:
for d in data_per_decade:
    print(len(d))

10
32
63
410
1534
3462
6601
20160
82622
161255
60036


In [91]:
from itertools import islice

for k, v in islice(data_per_decade[-1].items(), 2):
    print(k, v)

W1019189097 {'title': 'canonical correlation forests', 'publication_year': 2022, 'domain': 'Physical Sciences', 'field': 'Computer Science', 'topic': 'Neural Networks and Applications', 'abstract': 'We introduce canonical correlation forests (CCFs), a new decision tree ensemble method for classification and regression. Individual canonical correlation trees are binary decision trees with hyperplane splits based on local canonical correlation coefficients calculated during training. Unlike axis-aligned alternatives, the decision surfaces of CCFs are not restricted to the coordinate system of the inputs features and therefore more naturally represent data with correlated inputs. CCFs naturally accommodate multiple outputs, provide a similar computational complexity to random forests, and inherit their impressive robustness to the choice of input parameters. As part of the CCF training algorithm, we also introduce projection bootstrapping, a novel alternative to bagging for oblique decisi

In [34]:
def merge_title_and_abstract(entry:dict[str,str]):
    if isinstance(entry['abstract'], str):
        if isinstance(entry['title'], str):
            return entry['title'] + "\n" + entry['abstract']
        else:
            return entry['abstract']
    elif isinstance(entry['title'], str): 
        return entry['title']
    else:
        return None

In [102]:
import re
import unicodedata

In [90]:
def clean_openalex_text(text: str) -> str:
    # Unicode normalization
    text = unicodedata.normalize("NFKC", text)

    # Remove zero-width and BOM chars
    text = re.sub(r"[\u200b\u200c\u200d\ufeff]", "", text)

    # Remove LaTeX math blocks
    text = re.sub(r"\$[^$]+\$", " ", text)

    # Remove LaTeX commands (\alpha, \mathbb{R}, etc.)
    text = re.sub(r"\\[a-zA-Z]+(\{[^}]*\})?", " ", text)

    # Replace math operators & symbols with space
    text = re.sub(r"[∑∂≈⊗⊕≤≥≠∞√±×÷]", " ", text)

    # Collapse repeated punctuation
    text = re.sub(r"([=+\-*_]){2,}", " ", text)

    # Keep only letters, numbers, basic punctuation
    text = re.sub(r"[^0-9A-Za-zÀ-ÿ.,;:()\-\s]", " ", text)

    # Normalize whitespace
    text = re.sub(r"\s+", " ", text).strip()

    return text

In [105]:
corpus_per_decade = list()

for d in data_per_decade:
    txts = [merge_title_and_abstract(v) for v in d.values()]
    txts = [clean_openalex_text(entry.lower()) for entry in txts if isinstance(entry, str)]
    txts_split = [entry.split(" ") for entry in txts if isinstance(entry, str)]
    corpus_per_decade.append(txts_split)
    print(len(txts_split))

10
32
63
410
1534
3462
6601
20156
82605
161251
60036


In [106]:
corpus = [txt for txts in corpus_per_decade for txt in txts]

In [121]:
corpus[-1][:10]

['edge',
 'label',
 'inference',
 'in',
 'generalized',
 'stochastic',
 'block',
 'models:',
 'from',
 'spectral']

In [ ]:
%%time

#corpus: List[List[str]]
bigram = Phrases(
    corpus,
    min_count=5,
    threshold=10,
    delimiter="_",   # <-- str, not bytes
)
bigram_phraser = Phraser(bigram)

corpus_bi = [bigram_phraser[doc] for doc in corpus]

trigram = Phrases(
    corpus_bi,
    min_count=5,
    threshold=10,
    delimiter="_",
)
trigram_phraser = Phraser(trigram)

corpus_tri = [trigram_phraser[doc] for doc in corpus_bi]

In [52]:
corpus_bi

[['matroids', 'as', 'the', 'study', 'of', 'geometrical', 'configurations'],
 ['effects',
  'of',
  'electron-electron',
  'and',
  'electron-phonon',
  'interactions',
  'on',
  'the',
  'one-electron',
  'states',
  'of',
  'solids'],
 ['the', 'length', 'of', 'vectors', 'in', 'representation', 'spaces'],
 ['stratifications',
  'and',
  'mappings††ams',
  '(mos)',
  '1970',
  'subject',
  'classification:',
  '58c25.‡‡this',
  'chapter',
  'is',
  'based_on',
  'lectures',
  'i',
  'gave',
  'at',
  'the',
  'dynamical_systems',
  'conference.',
  'i',
  'wrote',
  'up',
  'these',
  'lectures',
  'partly',
  'during',
  'a',
  'stay',
  'at',
  'impa.',
  'i',
  'have',
  'developed',
  'these',
  'ideas',
  'over',
  'a',
  'period',
  'of',
  'years',
  'at',
  'princeton,',
  'ihes,',
  'harvard,',
  'and',
  'while',
  'supported_by',
  'a',
  'sloan',
  'fellowship',
  'and',
  'nsf',
  'grant',
  'gp-9566.i',
  'would',
  'like',
  'to',
  'thank',
  'all',
  'these',
  'institu

In [157]:
G = nx.DiGraph()

for src, targets in id_to_cited_ids.items():
    for tgt in targets:
        G.add_edge(src, tgt)
        
nx.set_node_attributes(G, id_to_attrs)

In [158]:
len(G.nodes)

2478

In [159]:
sinks = [n for n in G.nodes if G.out_degree(n) == 0]
sources = [n for n in G.nodes if G.in_degree(n) == 0]

In [160]:
len(sources)

874

In [169]:
G.node

{}

In [191]:
topics = [attrs.get("topic") if attrs.get("topic") is not None else "None" for _, attrs in G.nodes(data=True)]
domains = [attrs.get("domain") if attrs.get("domain") is not None else "None" for _, attrs in G.nodes(data=True)]
fields = [attrs.get("field") if attrs.get("field") is not None else "None" for _, attrs in G.nodes(data=True)]

In [203]:
Counter(topics).most_common(30)

[('None', 625),
 ('Particle physics theoretical and experimental studies', 75),
 ('Quantum Chromodynamics and Particle Interactions', 52),
 ('Cosmology and Gravitation Theories', 49),
 ('Black Holes and Theoretical Physics', 48),
 ('Matrix Theory and Algorithms', 43),
 ('Rings, Modules, and Algebras', 39),
 ('Electron and X-Ray Spectroscopy Techniques', 38),
 ('Mathematical Dynamics and Fractals', 32),
 ('Optimization and Variational Analysis', 28),
 ('Advanced Statistical Methods and Models', 26),
 ('Holomorphic and Operator Theory', 25),
 ('Spectral Theory in Mathematical Physics', 25),
 ('Quantum Mechanics and Applications', 24),
 ('Geometric Analysis and Curvature Flows', 24),
 ('Atomic and Molecular Physics', 22),
 ('High-pressure geophysics and materials', 22),
 ('Advanced Algebra and Geometry', 21),
 ('Advanced Topics in Algebra', 21),
 ('Economic theories and models', 21),
 ('Stability and Controllability of Differential Equations', 21),
 ('advanced mathematical theories', 21),

In [193]:
Counter(domains).most_common(10)

[('Physical Sciences', 1623),
 ('None', 625),
 ('Social Sciences', 162),
 ('Life Sciences', 55),
 ('Health Sciences', 13)]

In [194]:
Counter(fields).most_common(20)

[('None', 625),
 ('Physics and Astronomy', 528),
 ('Mathematics', 500),
 ('Computer Science', 214),
 ('Engineering', 185),
 ('Materials Science', 95),
 ('Economics, Econometrics and Finance', 89),
 ('Earth and Planetary Sciences', 59),
 ('Decision Sciences', 39),
 ('Chemistry', 23),
 ('Agricultural and Biological Sciences', 20),
 ('Neuroscience', 19),
 ('Environmental Science', 17),
 ('Business, Management and Accounting', 15),
 ('Biochemistry, Genetics and Molecular Biology', 15),
 ('Medicine', 11),
 ('Social Sciences', 9),
 ('Psychology', 7),
 ('Arts and Humanities', 3),
 ('Energy', 2)]

In [195]:
refs = [t for tgts in id_to_cited_ids.values() for t in tgts]
len(refs)

5163

In [ ]:
check = []
for r in refs:
    check.append(r in to_nodes)
    
Counter(check)